# This is My Webscrapping i was unable to import request libirary in jupyterlite so i used colab

In [1]:
import pandas as pd

# Load the ratings dataset
ratings = pd.read_csv('ratings.csv')

# Group user ratings based on movieId and apply aggregation operations
grouped_ratings = ratings.groupby('movieId').agg({'rating': ['count', 'mean']})

# Resetting index to have movieId as a column instead of index
grouped_ratings = grouped_ratings.reset_index()

# Load the movies dataset
movies = pd.read_csv('movies.csv')

# Perform inner join on the movies and grouped_ratings dataframes
merged_data = pd.merge(movies, grouped_ratings, on='movieId', how='inner')

# Rename the columns for clarity
merged_data.columns = ['movieId', 'title', 'genres', 'rating_count', 'avg_rating']

# Filter only those movies with more than 50 user ratings
filtered_data = merged_data[merged_data['rating_count'] > 50]

# Display the filtered data
print(filtered_data.head())

    movieId                           title  \
0         1                Toy Story (1995)   
1         2                  Jumanji (1995)   
5         6                     Heat (1995)   
9        10                GoldenEye (1995)   
10       11  American President, The (1995)   

                                         genres  rating_count  avg_rating  
0   Adventure|Animation|Children|Comedy|Fantasy           188    3.933511  
1                    Adventure|Children|Fantasy            94    3.398936  
5                         Action|Crime|Thriller            88    3.943182  
9                     Action|Adventure|Thriller           116    3.478448  
10                         Comedy|Drama|Romance            59    3.720339  


<ipython-input-1-4ba8319a1f66>:16: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  merged_data = pd.merge(movies, grouped_ratings, on='movieId', how='inner')
<ipython-input-1-4ba8319a1f66>:16: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_data = pd.merge(movies, grouped_ratings, on='movieId', how='inner')


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Load links data
links = pd.read_csv('links.csv')

# Assuming you have a DataFrame named filtered_data
# This DataFrame should contain movies with more than 50 user ratings
# Extract IMDB IDs from the links DataFrame
imdb_ids = filtered_data.merge(links[['movieId', 'imdbId']], on='movieId')['imdbId']

# Function to scrape reviews for a given IMDB ID
def scrape_reviews(imdb_id):
    imdb_url = f'https://www.imdb.com/title/tt0{imdb_id}/reviews'
    print(imdb_url)
    response = requests.get(imdb_url)
    print(response)

    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the parent container for reviews
        review_container = soup.find('div', class_='lister-list')

        if review_container:
            # Extract individual reviews
            reviews = review_container.find_all('div', class_='text show-more__control')

            # Print or store each review
            for index, review in enumerate(reviews, 1):
                print(f"Review {index}: {review.text.strip()}\n")
        else:
            print("Review container not found on the page.")
    else:
        print(f"Failed to retrieve reviews for {imdb_id}")

# Scrape reviews for each IMDB ID
for imdb_id in imdb_ids:
    print(f"Scraping reviews for IMDB ID: {imdb_id}")
    scrape_reviews(imdb_id)
    print("\n" + "=" * 50 + "\n")


Scraping reviews for IMDB ID: 114709
https://www.imdb.com/title/tt0114709/reviews
<Response [200]>
Review 1: Andy's toys live a reasonable life of fun and peace, their only worries are birthdays and Christmases, when new toys could easily replace those already there.  One such birthday Andy's top toy, Woody the cowboy, finds himself in direct competition with Andy's new Buzz Lightyear doll.  When rivalries boil over Woody tries to hide Buzz down the side of the bed but accidentally pushes him out the window, the other tops expel Woody, and he leaves with no choice but to find Buzz and return him to the house.  But with only two days before Andy moves house, time is of the essence.Given how often the same mix of animation, wit, jokes and kids humour has been used since Toy Story (Ice Age, Monsters Inc, Bugs Life) it is easy to forget how refreshing it was when it first came out.  I have just watched it again and it is dating a little in comparison to more recent twists on the formula.  

KeyboardInterrupt: 